In [6]:
from dataclasses import dataclass
import numpy as np
import pandas as pd
import torch
import math
import matplotlib.pyplot as plt

In [7]:

def format_results(label, best_numbers):
    #Formats weekly submission form 
    print(label)
    print(np.array2string(np.array(best_numbers), precision=6, separator='-', floatmode='fixed', formatter={'float': '{:0.6f}'.format}))
    
def load_and_process_data(function_num=1, num_weeks=1):
    # Load initial data
    def load_data(i):
        X = np.load(f'initial_data/function_{i}/initial_inputs.npy')
        y = np.load(f'initial_data/function_{i}/initial_outputs.npy')
        return X, y

    def load_data_2(i):
        X = np.load(f'initial_data2/function_{i}/initial_inputs.npy')
        y = np.load(f'initial_data2/function_{i}/initial_outputs.npy')
        return X, y
        
    X, y = load_data(function_num)
    dimension = X.shape[1]
    print(f"Shape of initial X: {X.shape}")
    print(f"Shape of initial y: {y.shape}")

    # Load and combine weekly data
    for week in range(1, num_weeks + 1):
        results_dir = f'results/week{week}'
        X_week = np.load(f'{results_dir}/f{function_num}.npy')
        y_week = np.load(f'{results_dir}/f_y{function_num}.npy')
        
        # Combine data
        X = np.vstack((X, X_week))
        y = np.concatenate((y, y_week))
        
        print(f"After week {week}:")
        print(f"Shape of X: {X.shape}")
        print(f"Shape of y: {y.shape}")

    print(f"\nadded new data Batch week20")
    X2, y2 = load_data(function_num)
    #dimension = X2.shape[1]
    print(f"Shape of initial X2: {X2.shape}")
    print(f"Shape of initial y2: {y2.shape}")
    X = np.vstack((X, X2))
    y = np.concatenate((y, y2))

    # Create DataFrame
    column_names = [f'X{i+1}' for i in range(X.shape[1])]
    df = pd.DataFrame(X, columns=column_names)
    df['y'] = y
    
    # Find maximum y value and corresponding row
    max_y = df['y'].max()
    max_row = df.loc[df['y'] == max_y]

   
    
    print(f"\nFinal DataFrame shape: {df.shape}")
    print(df)
    print(f"\nThe maximum y is: {max_y}")
    print("\nThe row with the max y is:")
    print(max_row)
    
    return X, y, df, max_y, max_row.iloc[0]

# Example usage
#function_num = 1
#num_weeks = 2  # Process data for 3 weeks
#X, y, df, max_y, max_row = load_and_process_data(function_num, num_weeks)

In [8]:
function_num = 1
num_week = 9

X, y, df, max_y, max_row = load_and_process_data(function_num, num_week)

Shape of initial X: (10, 2)
Shape of initial y: (10,)
After week 1:
Shape of X: (11, 2)
Shape of y: (11,)
After week 2:
Shape of X: (12, 2)
Shape of y: (12,)
After week 3:
Shape of X: (13, 2)
Shape of y: (13,)
After week 4:
Shape of X: (14, 2)
Shape of y: (14,)
After week 5:
Shape of X: (15, 2)
Shape of y: (15,)
After week 6:
Shape of X: (16, 2)
Shape of y: (16,)
After week 7:
Shape of X: (17, 2)
Shape of y: (17,)
After week 8:
Shape of X: (18, 2)
Shape of y: (18,)
After week 9:
Shape of X: (19, 2)
Shape of y: (19,)

added new data Batch week20
Shape of initial X2: (10, 2)
Shape of initial y2: (10,)

Final DataFrame shape: (29, 3)
          X1        X2              y
0   0.319404  0.762959   1.322677e-79
1   0.574329  0.879898   1.033078e-46
2   0.731024  0.733000   7.710875e-16
3   0.840353  0.264732  3.341771e-124
4   0.650114  0.681526  -3.606063e-03
5   0.410437  0.147554  -2.159249e-54
6   0.312691  0.078723  -2.089093e-91
7   0.683418  0.861057   2.535001e-40
8   0.082507  0.403

##  BayesianOptimizer class 

In [13]:
import torch
import gpytorch
import numpy as np
from gpytorch.models import ExactGP
from gpytorch.likelihoods import GaussianLikelihood
from gpytorch.mlls import ExactMarginalLogLikelihood
from gpytorch.kernels import RBFKernel, ScaleKernel
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import norm
device = torch.device("cuda:0") # Uncomment this to run on GPU

class GPModel(ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(GPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = ScaleKernel(RBFKernel(ard_num_dims=train_x.shape[1]))

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

class BayesianOptimizer:
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.scaler = MinMaxScaler()
        self.y_scaled = torch.tensor(self.scaler.fit_transform(y.reshape(-1, 1)).flatten(), dtype=torch.float32)
        self.likelihood = GaussianLikelihood()
        self.model = GPModel(self.X, self.y_scaled, self.likelihood)
        
    def train(self, training_iter=400, lr=0.1):
        self.model.train()
        self.likelihood.train()
        optimizer = torch.optim.Adam(self.model.parameters(), lr=lr)
        mll = ExactMarginalLogLikelihood(self.likelihood, self.model)
        
        for i in range(training_iter):
            optimizer.zero_grad()
            output = self.model(self.X)
            loss = -mll(output, self.y_scaled)
            loss.backward()
            if i % 20 == 19:
                print(f'Iter {i + 1}/{training_iter} - Loss: {loss.item():.4f}   Noise: {self.model.likelihood.noise.item():.4f}')
            optimizer.step()
        
        self.model.eval()
        self.likelihood.eval()
    
    @staticmethod
    def create_grid(X, num_points=100, hypercube_size=None, X_ini=None):
        num_vars = X.shape[1]
        
        if hypercube_size is not None:
            if X_ini is None:
                center = X.mean(axis=0)
            else:
                center = X_ini
            
            ranges = []
            for i in range(num_vars):
                lower_bound = max(0, center[i] - hypercube_size/2)
                upper_bound = min(1, center[i] + hypercube_size/2)
                ranges.append(np.linspace(lower_bound, upper_bound, num_points))
        else:
            ranges = [np.linspace(X[:, i].min(), X[:, i].max(), num_points) for i in range(num_vars)]
        
        grid = np.array(np.meshgrid(*ranges)).T.reshape(-1, num_vars)
        return grid
    
    @staticmethod
    def expected_improvement(mean, sigma, best_f, xi=0.01):
        with torch.no_grad():
            z = (mean - best_f - xi) / sigma
            normal = torch.distributions.Normal(torch.zeros_like(z), torch.ones_like(z))
            ei = (mean - best_f - xi) * normal.cdf(z) + sigma * normal.log_prob(z).exp()
        return ei
    
    def optimize(self, num_points=50, hypercube_size=0.5, X_ini=None):
        if X_ini is None:
            X_ini = self.X[self.y_scaled.argmax()]
        
        grid = self.create_grid(self.X.numpy(), num_points, hypercube_size, X_ini.numpy())
        grid_tensor = torch.tensor(grid, dtype=torch.float32)
        
        with torch.no_grad(), gpytorch.settings.fast_pred_var():
            predictions = self.likelihood(self.model(grid_tensor))
            mean = predictions.mean
            sigma = predictions.stddev
        
        best_f = self.y_scaled.max().item()
        ei = self.expected_improvement(mean, sigma, best_f)
        
        max_ei_value, max_ei_index = ei.max(0)
        max_ei_input = grid[max_ei_index]
        max_ei_prediction = mean[max_ei_index].item()
        max_ei_prediction_original = self.scaler.inverse_transform([[max_ei_prediction]])[0][0]
        
        actual_max_value = self.y_scaled.max().item()
        actual_max_value_original = self.scaler.inverse_transform([[actual_max_value]])[0][0]
        actual_max_index = self.y_scaled.argmax().item()
        actual_max_input = self.X[actual_max_index].numpy()
        
        results = {
            "max_ei_input": max_ei_input,
            "max_ei_prediction": max_ei_prediction_original,
            "max_ei_value": max_ei_value.item(),
            "actual_max_input": actual_max_input,
            "actual_max_value": actual_max_value_original
        }
        
        return results

# Example usage:
# optimizer = BayesianOptimizer(X, y)
# optimizer.train()
# results = optimizer.optimize()
# print(results)

In [15]:
import numpy as np
import torch
from sklearn.preprocessing import MinMaxScaler
#from bayesian_optimizer import BayesianOptimizer

# Scale y values
scaler = MinMaxScaler()
y_scaled = scaler.fit_transform(y.reshape(-1, 1)).flatten()

# Convert numpy arrays to torch tensors
X = torch.tensor(X, dtype=torch.float32)
y_scaled = torch.tensor(y_scaled, dtype=torch.float32)

# Create and use the optimizer
optimizer = BayesianOptimizer(X, y_scaled)
optimizer.train(training_iter=500)
results = optimizer.optimize(num_points=50, hypercube_size=0.1)

print("Optimization Results:")
print(f"Best point found: {results['max_ei_input']}")
print(f"Predicted value at best point (scaled): {results['max_ei_prediction']}")
print(f"Predicted value at best point (original scale): {scaler.inverse_transform([[results['max_ei_prediction']]])[0][0]}")
print(f"Expected Improvement: {results['max_ei_value']}")
print(f"\nActual best point in training data: {results['actual_max_input']}")
print(f"Actual best value in training data (scaled): {results['actual_max_value']}")
print(f"Actual best value in training data (original scale): {scaler.inverse_transform([[results['actual_max_value']]])[0][0]}")
print('\nFunction Number:',function_num)
format_results('Next query:', results['max_ei_input'])

C:\Users\claud\AppData\Local\Temp\ipykernel_32040\2251220785.py:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.X = torch.tensor(X, dtype=torch.float32)


Iter 20/500 - Loss: 0.1183   Noise: 0.1372
Iter 40/500 - Loss: -0.2234   Noise: 0.0285
Iter 60/500 - Loss: -0.2600   Noise: 0.0345
Iter 80/500 - Loss: -0.2751   Noise: 0.0339
Iter 100/500 - Loss: -0.2826   Noise: 0.0331
Iter 120/500 - Loss: -0.2857   Noise: 0.0331
Iter 140/500 - Loss: -0.2870   Noise: 0.0330
Iter 160/500 - Loss: -0.2877   Noise: 0.0329
Iter 180/500 - Loss: -0.2876   Noise: 0.0330
Iter 200/500 - Loss: -0.2884   Noise: 0.0329
Iter 220/500 - Loss: -0.2887   Noise: 0.0329
Iter 240/500 - Loss: -0.2888   Noise: 0.0329
Iter 260/500 - Loss: -0.2878   Noise: 0.0329
Iter 280/500 - Loss: -0.2891   Noise: 0.0328
Iter 300/500 - Loss: -0.2891   Noise: 0.0328
Iter 320/500 - Loss: -0.2892   Noise: 0.0328
Iter 340/500 - Loss: -0.2892   Noise: 0.0328
Iter 360/500 - Loss: -0.2890   Noise: 0.0329
Iter 380/500 - Loss: -0.2893   Noise: 0.0328
Iter 400/500 - Loss: -0.2894   Noise: 0.0328
Iter 420/500 - Loss: -0.2894   Noise: 0.0328
Iter 440/500 - Loss: -0.2883   Noise: 0.0329
Iter 460/500 - 

In [17]:
function_num = 2
num_week = 9

X, y, df, max_y, max_row = load_and_process_data(function_num, num_week)

Shape of initial X: (10, 2)
Shape of initial y: (10,)
After week 1:
Shape of X: (11, 2)
Shape of y: (11,)
After week 2:
Shape of X: (12, 2)
Shape of y: (12,)
After week 3:
Shape of X: (13, 2)
Shape of y: (13,)
After week 4:
Shape of X: (14, 2)
Shape of y: (14,)
After week 5:
Shape of X: (15, 2)
Shape of y: (15,)
After week 6:
Shape of X: (16, 2)
Shape of y: (16,)
After week 7:
Shape of X: (17, 2)
Shape of y: (17,)
After week 8:
Shape of X: (18, 2)
Shape of y: (18,)
After week 9:
Shape of X: (19, 2)
Shape of y: (19,)

added new data Batch week20
Shape of initial X2: (10, 2)
Shape of initial y2: (10,)

Final DataFrame shape: (29, 3)
          X1        X2         y
0   0.665800  0.123969  0.538996
1   0.877791  0.778628  0.420586
2   0.142699  0.349005 -0.065624
3   0.845275  0.711120  0.293993
4   0.454647  0.290455  0.214965
5   0.577713  0.771973  0.023106
6   0.438166  0.685018  0.244619
7   0.341750  0.028698  0.038749
8   0.338648  0.213867 -0.013858
9   0.702637  0.926564  0.61120

In [19]:

# Scale y values
scaler = MinMaxScaler()
y_scaled = scaler.fit_transform(y.reshape(-1, 1)).flatten()
X = torch.tensor(X, dtype=torch.float32)
y_scaled = torch.tensor(y_scaled, dtype=torch.float32)

# Create and use the optimizer
optimizer = BayesianOptimizer(X, y_scaled)
optimizer.train(training_iter=200)
results = optimizer.optimize(num_points=50, hypercube_size=0.15)

print("Optimization Results:")
print(f"Best point found: {results['max_ei_input']}")
print(f"Predicted value at best point (scaled): {results['max_ei_prediction']}")
print(f"Predicted value at best point (original scale): {scaler.inverse_transform([[results['max_ei_prediction']]])[0][0]}")
print(f"Expected Improvement: {results['max_ei_value']}")
print(f"\nActual best point in training data: {results['actual_max_input']}")
print(f"Actual best value in training data (scaled): {results['actual_max_value']}")
print(f"Actual best value in training data (original scale): {scaler.inverse_transform([[results['actual_max_value']]])[0][0]}")
print('\nFunction Number:',function_num)
format_results('Next query:', results['max_ei_input'])

C:\Users\claud\AppData\Local\Temp\ipykernel_32040\2251220785.py:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.X = torch.tensor(X, dtype=torch.float32)


Iter 20/200 - Loss: 0.2779   Noise: 0.1403
Iter 40/200 - Loss: -0.2708   Noise: 0.0328
Iter 60/200 - Loss: -0.7913   Noise: 0.0052
Iter 80/200 - Loss: -0.9004   Noise: 0.0017
Iter 100/200 - Loss: -0.9132   Noise: 0.0021
Iter 120/200 - Loss: -0.9139   Noise: 0.0022
Iter 140/200 - Loss: -0.9145   Noise: 0.0021
Iter 160/200 - Loss: -0.9148   Noise: 0.0022
Iter 180/200 - Loss: -0.9151   Noise: 0.0022
Iter 200/200 - Loss: -0.9153   Noise: 0.0022
Optimization Results:
Best point found: [0.68622493 0.86377602]
Predicted value at best point (scaled): 0.9274374842643738
Predicted value at best point (original scale): 0.5856691197321947
Expected Improvement: 0.001004224643111229

Actual best point in training data: [0.693878 0.938776]
Actual best value in training data (scaled): 1.0
Actual best value in training data (original scale): 0.636626128478489

Function Number: 2
Next query:
[0.686225-0.863776]


In [21]:
function_num = 3
num_week = 9

X, y, df, max_y, max_row = load_and_process_data(function_num, num_week)

Shape of initial X: (15, 3)
Shape of initial y: (15,)
After week 1:
Shape of X: (16, 3)
Shape of y: (16,)
After week 2:
Shape of X: (17, 3)
Shape of y: (17,)
After week 3:
Shape of X: (18, 3)
Shape of y: (18,)
After week 4:
Shape of X: (19, 3)
Shape of y: (19,)
After week 5:
Shape of X: (20, 3)
Shape of y: (20,)
After week 6:
Shape of X: (21, 3)
Shape of y: (21,)
After week 7:
Shape of X: (22, 3)
Shape of y: (22,)
After week 8:
Shape of X: (23, 3)
Shape of y: (23,)
After week 9:
Shape of X: (24, 3)
Shape of y: (24,)

added new data Batch week20
Shape of initial X2: (15, 3)
Shape of initial y2: (15,)

Final DataFrame shape: (39, 4)
          X1        X2        X3         y
0   0.171525  0.343917  0.248737 -0.112122
1   0.242114  0.644074  0.272433 -0.087963
2   0.534906  0.398501  0.173389 -0.111415
3   0.492581  0.611593  0.340176 -0.034835
4   0.134622  0.219917  0.458206 -0.048008
5   0.345523  0.941360  0.269363 -0.110621
6   0.151837  0.439991  0.990882 -0.398926
7   0.645503  0.3

In [23]:

# Scale y values
scaler = MinMaxScaler()
y_scaled = scaler.fit_transform(y.reshape(-1, 1)).flatten()
X = torch.tensor(X, dtype=torch.float32)
y_scaled = torch.tensor(y_scaled, dtype=torch.float32)

# Create and use the optimizer
optimizer = BayesianOptimizer(X, y_scaled)
optimizer.train(training_iter=200)
results = optimizer.optimize(num_points=50, hypercube_size=0.2)

print("Optimization Results:")
print(f"Best point found: {results['max_ei_input']}")
print(f"Predicted value at best point (scaled): {results['max_ei_prediction']}")
print(f"Predicted value at best point (original scale): {scaler.inverse_transform([[results['max_ei_prediction']]])[0][0]}")
print(f"Expected Improvement: {results['max_ei_value']}")
print(f"\nActual best point in training data: {results['actual_max_input']}")
print(f"Actual best value in training data (scaled): {results['actual_max_value']}")
print(f"Actual best value in training data (original scale): {scaler.inverse_transform([[results['actual_max_value']]])[0][0]}")
print('\nFunction Number:',function_num)
format_results('Next query:', results['max_ei_input'])

C:\Users\claud\AppData\Local\Temp\ipykernel_32040\2251220785.py:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.X = torch.tensor(X, dtype=torch.float32)


Iter 20/200 - Loss: 0.1234   Noise: 0.1371
Iter 40/200 - Loss: -0.4672   Noise: 0.0238
Iter 60/200 - Loss: -1.0309   Noise: 0.0045
Iter 80/200 - Loss: -1.4984   Noise: 0.0008
Iter 100/200 - Loss: -1.6552   Noise: 0.0003
Iter 120/200 - Loss: -1.6770   Noise: 0.0002
Iter 140/200 - Loss: -1.7783   Noise: 0.0002
Iter 160/200 - Loss: -1.8284   Noise: 0.0001
Iter 180/200 - Loss: -1.8524   Noise: 0.0001
Iter 200/200 - Loss: -1.8651   Noise: 0.0001
Optimization Results:
Best point found: [0.85510198 0.56938788 0.49183715]
Predicted value at best point (scaled): 0.9929190278053284
Predicted value at best point (original scale): -0.004410183142779513
Expected Improvement: 0.0008205580525100231

Actual best point in training data: [0.755102 0.612245 0.44898 ]
Actual best value in training data (scaled): 1.0
Actual best value in training data (original scale): -0.001021473822535214

Function Number: 3
Next query:
[0.855102-0.569388-0.491837]


In [36]:
function_num = 4
num_week = 9

X, y, df, max_y, max_row = load_and_process_data(function_num, num_week)

Shape of initial X: (30, 4)
Shape of initial y: (30,)
After week 1:
Shape of X: (31, 4)
Shape of y: (31,)
After week 2:
Shape of X: (32, 4)
Shape of y: (32,)
After week 3:
Shape of X: (33, 4)
Shape of y: (33,)
After week 4:
Shape of X: (34, 4)
Shape of y: (34,)
After week 5:
Shape of X: (35, 4)
Shape of y: (35,)
After week 6:
Shape of X: (36, 4)
Shape of y: (36,)
After week 7:
Shape of X: (37, 4)
Shape of y: (37,)
After week 8:
Shape of X: (38, 4)
Shape of y: (38,)
After week 9:
Shape of X: (39, 4)
Shape of y: (39,)

added new data Batch week20
Shape of initial X2: (30, 4)
Shape of initial y2: (30,)

Final DataFrame shape: (69, 5)
          X1        X2        X3        X4          y
0   0.896981  0.725628  0.175404  0.701694 -22.108288
1   0.889356  0.499588  0.539269  0.508783 -14.601397
2   0.250946  0.033693  0.145380  0.494932 -11.699932
3   0.346962  0.006250  0.760564  0.613024 -16.053765
4   0.124871  0.129770  0.384400  0.287076 -10.069633
..       ...       ...       ...     

In [37]:

# Scale y values
scaler = MinMaxScaler()
y_scaled = scaler.fit_transform(y.reshape(-1, 1)).flatten()
X = torch.tensor(X, dtype=torch.float32)
y_scaled = torch.tensor(y_scaled, dtype=torch.float32)

# Create and use the optimizer
optimizer = BayesianOptimizer(X, y_scaled)
optimizer.train(training_iter=400)
results = optimizer.optimize(num_points=50, hypercube_size=0.3)

print("Optimization Results:")
print(f"Best point found: {results['max_ei_input']}")
print(f"Predicted value at best point (scaled): {results['max_ei_prediction']}")
print(f"Predicted value at best point (original scale): {scaler.inverse_transform([[results['max_ei_prediction']]])[0][0]}")
print(f"Expected Improvement: {results['max_ei_value']}")
print(f"\nActual best point in training data: {results['actual_max_input']}")
print(f"Actual best value in training data (scaled): {results['actual_max_value']}")
print(f"Actual best value in training data (original scale): {scaler.inverse_transform([[results['actual_max_value']]])[0][0]}")
print('\nFunction Number:',function_num)
format_results('Next query:', results['max_ei_input'])

Iter 20/400 - Loss: 0.1329   Noise: 0.1372
Iter 40/400 - Loss: -0.6412   Noise: 0.0196


C:\Users\claud\AppData\Local\Temp\ipykernel_17416\2251220785.py:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.X = torch.tensor(X, dtype=torch.float32)


Iter 60/400 - Loss: -1.3535   Noise: 0.0029
Iter 80/400 - Loss: -1.8515   Noise: 0.0006
Iter 100/400 - Loss: -2.1023   Noise: 0.0002
Iter 120/400 - Loss: -2.1878   Noise: 0.0001
Iter 140/400 - Loss: -2.2177   Noise: 0.0001
Iter 160/400 - Loss: -2.2313   Noise: 0.0001
Iter 180/400 - Loss: -2.2390   Noise: 0.0001
Iter 200/400 - Loss: -2.2440   Noise: 0.0001
Iter 220/400 - Loss: -2.2475   Noise: 0.0001
Iter 240/400 - Loss: -2.2500   Noise: 0.0001
Iter 260/400 - Loss: -2.2520   Noise: 0.0001
Iter 280/400 - Loss: -2.2535   Noise: 0.0001
Iter 300/400 - Loss: -2.2548   Noise: 0.0001
Iter 320/400 - Loss: -2.2558   Noise: 0.0001
Iter 340/400 - Loss: -2.2566   Noise: 0.0001
Iter 360/400 - Loss: -2.2574   Noise: 0.0001
Iter 380/400 - Loss: -2.2580   Noise: 0.0001
Iter 400/400 - Loss: -2.2585   Noise: 0.0001
Optimization Results:
Best point found: [0.42163222 0.40081622 0.33142863 0.42734668]
Predicted value at best point (scaled): 0.9880874156951904
Predicted value at best point (original scale):

In [38]:
function_num = 5
num_week = 9

X, y, df, max_y, max_row = load_and_process_data(function_num, num_week)

Shape of initial X: (20, 4)
Shape of initial y: (20,)
After week 1:
Shape of X: (21, 4)
Shape of y: (21,)
After week 2:
Shape of X: (22, 4)
Shape of y: (22,)
After week 3:
Shape of X: (23, 4)
Shape of y: (23,)
After week 4:
Shape of X: (24, 4)
Shape of y: (24,)
After week 5:
Shape of X: (25, 4)
Shape of y: (25,)
After week 6:
Shape of X: (26, 4)
Shape of y: (26,)
After week 7:
Shape of X: (27, 4)
Shape of y: (27,)
After week 8:
Shape of X: (28, 4)
Shape of y: (28,)
After week 9:
Shape of X: (29, 4)
Shape of y: (29,)

added new data Batch week20
Shape of initial X2: (20, 4)
Shape of initial y2: (20,)

Final DataFrame shape: (49, 5)
          X1        X2        X3        X4            y
0   0.191447  0.038193  0.607418  0.414584    64.443440
1   0.758653  0.536518  0.656000  0.360342    18.301380
2   0.438350  0.804340  0.210245  0.151295     0.112940
3   0.706051  0.534192  0.264243  0.482088     4.210898
4   0.836478  0.193610  0.663893  0.785649   258.370525
5   0.683432  0.118663  0

In [39]:

# Scale y values
scaler = MinMaxScaler()
y_scaled = scaler.fit_transform(y.reshape(-1, 1)).flatten()
X = torch.tensor(X, dtype=torch.float32)
y_scaled = torch.tensor(y_scaled, dtype=torch.float32)

# Create and use the optimizer
optimizer = BayesianOptimizer(X, y_scaled)
optimizer.train(training_iter=500)
results = optimizer.optimize(num_points=40, hypercube_size=0.05)

print("Optimization Results:")
print(f"Best point found: {results['max_ei_input']}")
print(f"Predicted value at best point (scaled): {results['max_ei_prediction']}")
print(f"Predicted value at best point (original scale): {scaler.inverse_transform([[results['max_ei_prediction']]])[0][0]}")
print(f"Expected Improvement: {results['max_ei_value']}")
print(f"\nActual best point in training data: {results['actual_max_input']}")
print(f"Actual best value in training data (scaled): {results['actual_max_value']}")
print(f"Actual best value in training data (original scale): {scaler.inverse_transform([[results['actual_max_value']]])[0][0]}")
print('\nFunction Number:',function_num)
format_results('Next query:', results['max_ei_input'])

Iter 20/500 - Loss: 0.0873   Noise: 0.1360


C:\Users\claud\AppData\Local\Temp\ipykernel_17416\2251220785.py:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.X = torch.tensor(X, dtype=torch.float32)


Iter 40/500 - Loss: -0.6359   Noise: 0.0200
Iter 60/500 - Loss: -1.2213   Noise: 0.0033
Iter 80/500 - Loss: -1.4692   Noise: 0.0010
Iter 100/500 - Loss: -1.5587   Noise: 0.0005
Iter 120/500 - Loss: -1.7082   Noise: 0.0003
Iter 140/500 - Loss: -1.7916   Noise: 0.0002
Iter 160/500 - Loss: -1.8541   Noise: 0.0001
Iter 180/500 - Loss: -1.9179   Noise: 0.0001
Iter 200/500 - Loss: -1.9340   Noise: 0.0001
Iter 220/500 - Loss: -1.9424   Noise: 0.0001
Iter 240/500 - Loss: -1.9480   Noise: 0.0001
Iter 260/500 - Loss: -1.9529   Noise: 0.0001
Iter 280/500 - Loss: -1.9592   Noise: 0.0001
Iter 300/500 - Loss: -1.9676   Noise: 0.0001
Iter 320/500 - Loss: -1.9695   Noise: 0.0001
Iter 340/500 - Loss: -1.9707   Noise: 0.0001
Iter 360/500 - Loss: -1.9717   Noise: 0.0001
Iter 380/500 - Loss: -1.9724   Noise: 0.0001
Iter 400/500 - Loss: -1.9730   Noise: 0.0001
Iter 420/500 - Loss: -1.9735   Noise: 0.0001
Iter 440/500 - Loss: -1.9743   Noise: 0.0001
Iter 460/500 - Loss: -1.9745   Noise: 0.0001
Iter 480/500 

In [40]:
function_num = 6
num_week = 9

X, y, df, max_y, max_row = load_and_process_data(function_num, num_week)

Shape of initial X: (20, 5)
Shape of initial y: (20,)
After week 1:
Shape of X: (21, 5)
Shape of y: (21,)
After week 2:
Shape of X: (22, 5)
Shape of y: (22,)
After week 3:
Shape of X: (23, 5)
Shape of y: (23,)
After week 4:
Shape of X: (24, 5)
Shape of y: (24,)
After week 5:
Shape of X: (25, 5)
Shape of y: (25,)
After week 6:
Shape of X: (26, 5)
Shape of y: (26,)
After week 7:
Shape of X: (27, 5)
Shape of y: (27,)
After week 8:
Shape of X: (28, 5)
Shape of y: (28,)
After week 9:
Shape of X: (29, 5)
Shape of y: (29,)

added new data Batch week20
Shape of initial X2: (20, 5)
Shape of initial y2: (20,)

Final DataFrame shape: (49, 6)
          X1        X2        X3        X4        X5         y
0   0.728186  0.154693  0.732552  0.693997  0.056401 -0.714265
1   0.242384  0.844100  0.577809  0.679021  0.501953 -1.209955
2   0.729523  0.748106  0.679775  0.356552  0.671054 -1.672200
3   0.770620  0.114404  0.046780  0.648324  0.273549 -1.536058
4   0.618812  0.331802  0.187288  0.756238  0.

In [41]:

# Scale y values
scaler = MinMaxScaler()
y_scaled = scaler.fit_transform(y.reshape(-1, 1)).flatten()
X = torch.tensor(X, dtype=torch.float32)
y_scaled = torch.tensor(y_scaled, dtype=torch.float32)

# Create and use the optimizer
optimizer = BayesianOptimizer(X, y_scaled)
optimizer.train(training_iter=500)
results = optimizer.optimize(num_points=30, hypercube_size=0.2)

print("Optimization Results:")
print(f"Best point found: {results['max_ei_input']}")
print(f"Predicted value at best point (scaled): {results['max_ei_prediction']}")
print(f"Predicted value at best point (original scale): {scaler.inverse_transform([[results['max_ei_prediction']]])[0][0]}")
print(f"Expected Improvement: {results['max_ei_value']}")
print(f"\nActual best point in training data: {results['actual_max_input']}")
print(f"Actual best value in training data (scaled): {results['actual_max_value']}")
print(f"Actual best value in training data (original scale): {scaler.inverse_transform([[results['actual_max_value']]])[0][0]}")
print('\nFunction Number:',function_num)
format_results('Next query:', results['max_ei_input'])

Iter 20/500 - Loss: 0.1046   Noise: 0.1357


C:\Users\claud\AppData\Local\Temp\ipykernel_17416\2251220785.py:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.X = torch.tensor(X, dtype=torch.float32)


Iter 40/500 - Loss: -0.4963   Noise: 0.0210
Iter 60/500 - Loss: -1.0432   Noise: 0.0045
Iter 80/500 - Loss: -1.2872   Noise: 0.0012
Iter 100/500 - Loss: -1.4398   Noise: 0.0006
Iter 120/500 - Loss: -1.5742   Noise: 0.0003
Iter 140/500 - Loss: -1.6730   Noise: 0.0002
Iter 160/500 - Loss: -1.7183   Noise: 0.0001
Iter 180/500 - Loss: -1.7388   Noise: 0.0001
Iter 200/500 - Loss: -1.7543   Noise: 0.0001
Iter 220/500 - Loss: -1.7742   Noise: 0.0001
Iter 240/500 - Loss: -1.7829   Noise: 0.0001
Iter 260/500 - Loss: -1.7869   Noise: 0.0001
Iter 280/500 - Loss: -1.7895   Noise: 0.0001
Iter 300/500 - Loss: -1.7913   Noise: 0.0001
Iter 320/500 - Loss: -1.7927   Noise: 0.0001
Iter 340/500 - Loss: -1.7939   Noise: 0.0001
Iter 360/500 - Loss: -1.7948   Noise: 0.0001
Iter 380/500 - Loss: -1.7956   Noise: 0.0001
Iter 400/500 - Loss: -1.7962   Noise: 0.0001
Iter 420/500 - Loss: -1.7968   Noise: 0.0001
Iter 440/500 - Loss: -1.7972   Noise: 0.0001
Iter 460/500 - Loss: -1.7976   Noise: 0.0001
Iter 480/500 

In [42]:
function_num = 7
num_week = 9

X, y, df, max_y, max_row = load_and_process_data(function_num, num_week)

Shape of initial X: (30, 6)
Shape of initial y: (30,)
After week 1:
Shape of X: (31, 6)
Shape of y: (31,)
After week 2:
Shape of X: (32, 6)
Shape of y: (32,)
After week 3:
Shape of X: (33, 6)
Shape of y: (33,)
After week 4:
Shape of X: (34, 6)
Shape of y: (34,)
After week 5:
Shape of X: (35, 6)
Shape of y: (35,)
After week 6:
Shape of X: (36, 6)
Shape of y: (36,)
After week 7:
Shape of X: (37, 6)
Shape of y: (37,)
After week 8:
Shape of X: (38, 6)
Shape of y: (38,)
After week 9:
Shape of X: (39, 6)
Shape of y: (39,)

added new data Batch week20
Shape of initial X2: (30, 6)
Shape of initial y2: (30,)

Final DataFrame shape: (69, 7)
          X1        X2        X3        X4        X5        X6         y
0   0.272624  0.324495  0.897109  0.832951  0.154063  0.795864  0.604433
1   0.543003  0.924694  0.341567  0.646486  0.718440  0.343133  0.562753
2   0.090832  0.661529  0.065931  0.258577  0.963453  0.640265  0.007503
3   0.118867  0.615055  0.905816  0.855300  0.413631  0.585236  0.061

In [43]:

# Scale y values
scaler = MinMaxScaler()
y_scaled = scaler.fit_transform(y.reshape(-1, 1)).flatten()
X = torch.tensor(X, dtype=torch.float32)
y_scaled = torch.tensor(y_scaled, dtype=torch.float32)

# Create and use the optimizer
optimizer = BayesianOptimizer(X, y_scaled)
optimizer.train(training_iter=500)
results = optimizer.optimize(num_points=15, hypercube_size=0.2)

print("Optimization Results:")
print(f"Best point found: {results['max_ei_input']}")
print(f"Predicted value at best point (scaled): {results['max_ei_prediction']}")
print(f"Predicted value at best point (original scale): {scaler.inverse_transform([[results['max_ei_prediction']]])[0][0]}")
print(f"Expected Improvement: {results['max_ei_value']}")
print(f"\nActual best point in training data: {results['actual_max_input']}")
print(f"Actual best value in training data (scaled): {results['actual_max_value']}")
print(f"Actual best value in training data (original scale): {scaler.inverse_transform([[results['actual_max_value']]])[0][0]}")
print('\nFunction Number:',function_num)
format_results('Next query:', results['max_ei_input'])

C:\Users\claud\AppData\Local\Temp\ipykernel_17416\2251220785.py:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.X = torch.tensor(X, dtype=torch.float32)


Iter 20/500 - Loss: 0.1502   Noise: 0.1366
Iter 40/500 - Loss: -0.3631   Noise: 0.0255
Iter 60/500 - Loss: -0.9465   Noise: 0.0049
Iter 80/500 - Loss: -1.3574   Noise: 0.0010
Iter 100/500 - Loss: -1.6192   Noise: 0.0003
Iter 120/500 - Loss: -1.7424   Noise: 0.0002
Iter 140/500 - Loss: -1.7894   Noise: 0.0001
Iter 160/500 - Loss: -1.8084   Noise: 0.0001
Iter 180/500 - Loss: -1.8180   Noise: 0.0001
Iter 200/500 - Loss: -1.8236   Noise: 0.0001
Iter 220/500 - Loss: -1.8272   Noise: 0.0001
Iter 240/500 - Loss: -1.8300   Noise: 0.0001
Iter 260/500 - Loss: -1.8319   Noise: 0.0001
Iter 280/500 - Loss: -1.8335   Noise: 0.0001
Iter 300/500 - Loss: -1.8347   Noise: 0.0001
Iter 320/500 - Loss: -1.8357   Noise: 0.0001
Iter 340/500 - Loss: -1.8365   Noise: 0.0001
Iter 360/500 - Loss: -1.8367   Noise: 0.0001
Iter 380/500 - Loss: -1.8377   Noise: 0.0001
Iter 400/500 - Loss: -1.8382   Noise: 0.0001
Iter 420/500 - Loss: -1.8386   Noise: 0.0001
Iter 440/500 - Loss: -1.8390   Noise: 0.0001
Iter 460/500 - 

In [44]:
function_num = 8
num_week = 9

X, y, df, max_y, max_row = load_and_process_data(function_num, num_week)

Shape of initial X: (40, 8)
Shape of initial y: (40,)
After week 1:
Shape of X: (41, 8)
Shape of y: (41,)
After week 2:
Shape of X: (42, 8)
Shape of y: (42,)
After week 3:
Shape of X: (43, 8)
Shape of y: (43,)
After week 4:
Shape of X: (44, 8)
Shape of y: (44,)
After week 5:
Shape of X: (45, 8)
Shape of y: (45,)
After week 6:
Shape of X: (46, 8)
Shape of y: (46,)
After week 7:
Shape of X: (47, 8)
Shape of y: (47,)
After week 8:
Shape of X: (48, 8)
Shape of y: (48,)
After week 9:
Shape of X: (49, 8)
Shape of y: (49,)

added new data Batch week20
Shape of initial X2: (40, 8)
Shape of initial y2: (40,)

Final DataFrame shape: (89, 9)
          X1        X2        X3        X4        X5        X6        X7  \
0   0.604994  0.292215  0.908453  0.355506  0.201669  0.575338  0.310311   
1   0.178007  0.566223  0.994862  0.210325  0.320153  0.707909  0.635384   
2   0.009077  0.811626  0.520520  0.075687  0.265112  0.091652  0.592415   
3   0.506028  0.653730  0.363411  0.177981  0.093728  0.1

In [45]:

# Scale y values
scaler = MinMaxScaler()
y_scaled = scaler.fit_transform(y.reshape(-1, 1)).flatten()
X = torch.tensor(X, dtype=torch.float32)
y_scaled = torch.tensor(y_scaled, dtype=torch.float32)

# Create and use the optimizer
optimizer = BayesianOptimizer(X, y_scaled)
optimizer.train(training_iter=500)
results = optimizer.optimize(num_points=5, hypercube_size=0.1)

print("Optimization Results:")
print(f"Best point found: {results['max_ei_input']}")
print(f"Predicted value at best point (scaled): {results['max_ei_prediction']}")
print(f"Predicted value at best point (original scale): {scaler.inverse_transform([[results['max_ei_prediction']]])[0][0]}")
print(f"Expected Improvement: {results['max_ei_value']}")
print(f"\nActual best point in training data: {results['actual_max_input']}")
print(f"Actual best value in training data (scaled): {results['actual_max_value']}")
print(f"Actual best value in training data (original scale): {scaler.inverse_transform([[results['actual_max_value']]])[0][0]}")
print('\nFunction Number:',function_num)
format_results('Next query:', results['max_ei_input'])

Iter 20/500 - Loss: 0.0576   Noise: 0.1343


C:\Users\claud\AppData\Local\Temp\ipykernel_17416\2251220785.py:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.X = torch.tensor(X, dtype=torch.float32)


Iter 40/500 - Loss: -0.8143   Noise: 0.0176
Iter 60/500 - Loss: -1.5258   Noise: 0.0026
Iter 80/500 - Loss: -2.0595   Noise: 0.0005
Iter 100/500 - Loss: -2.3659   Noise: 0.0002
Iter 120/500 - Loss: -2.4849   Noise: 0.0001
Iter 140/500 - Loss: -2.5292   Noise: 0.0001
Iter 160/500 - Loss: -2.5496   Noise: 0.0001
Iter 180/500 - Loss: -2.5614   Noise: 0.0001
Iter 200/500 - Loss: -2.5691   Noise: 0.0001
Iter 220/500 - Loss: -2.5744   Noise: 0.0001
Iter 240/500 - Loss: -2.5785   Noise: 0.0001
Iter 260/500 - Loss: -2.5816   Noise: 0.0001
Iter 280/500 - Loss: -2.5842   Noise: 0.0001
Iter 300/500 - Loss: -2.5864   Noise: 0.0001
Iter 320/500 - Loss: -2.5882   Noise: 0.0001
Iter 340/500 - Loss: -2.5895   Noise: 0.0001
Iter 360/500 - Loss: -2.5909   Noise: 0.0001
Iter 380/500 - Loss: -2.5921   Noise: 0.0001
Iter 400/500 - Loss: -2.5932   Noise: 0.0001
Iter 420/500 - Loss: -2.5941   Noise: 0.0001
Iter 440/500 - Loss: -2.5948   Noise: 0.0001
Iter 460/500 - Loss: -2.5957   Noise: 0.0001
Iter 480/500 